https://data.novascotia.ca/Health-and-Wellness/Action-for-Health/m9ng-y7cu/about_data  
https://public.tableau.com/app/profile/nova.scotia.health/viz/ActionforHealth-PublicReporting/Overview

In [1]:
import pandas as pd

Function to replace nulls

In [2]:
def replaceNulls(column, value):
    """Replaces nulls. Input column with nulls and value to replace them with.
    """
    df[column] = df[column].fillna(value)

Functions to create lookups and map them for fact table

In [3]:
def lup(ser):
    """Pass in a series to the function. The function will return two dictionaries in a tuple
    Based on the unique fields in the series:
    1) a dictionary to be used as the basis for a dimension,
    2) a lookup to be used to encode the fact table
    """
    ser_dict = dict(enumerate(ser.unique(), start=1))
    ser_lup = {value: key for key, value in ser_dict.items()}
    return ser_dict, ser_lup

def lup_map(df, column_name):
    """Maps the lookups created in the lup(ser) function.
    Adds '_ID' to column name.
    """
    ser = df[column_name]
    ser_dict, ser_lup = lup(ser)
    df[column_name+'_ID'] = ser.map(ser_lup)
    return ser_dict, ser_lup

Functions for dim builds

In [4]:
def build_cal_dim(start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    cal_df = pd.DataFrame(date_range, columns=['date'])
    cal_df['Year'] = cal_df['date'].dt.year
    cal_df['Month'] = cal_df['date'].dt.month
    cal_df['Day'] = cal_df['date'].dt.day
    return cal_df

def df_dims(df, ser1, ser2):
    """
    Returns a new dateframe with selected columns and drops duplicates.
    - df: The original df.
    - ser1: First column for your new df and is the subset for duplicate removal.
    - ser2: Second column for your new df.
    """
    return df[[ser1, ser2]].drop_duplicates(subset=ser1)

In [5]:
df = pd.read_csv('./Data/Action_for_Health_20240327.csv')
df

,Zone,Hospital,Type,Date,Measure Name,Actual,CTAS
0,Central,Dartmouth General Hospital,Regional Hospital,2021-01-01,Acute Occupancy,93.478,NaN
1,Central,Dartmouth General Hospital,Regional Hospital,2021-04-01,Acute Occupancy,80.797,NaN
2,Central,Dartmouth General Hospital,Regional Hospital,2021-05-01,Acute Occupancy,78.612,NaN
3,Central,Dartmouth General Hospital,Regional Hospital,2021-06-01,Acute Occupancy,75.525,NaN
4,Central,Dartmouth General Hospital,Regional Hospital,2021-07-01,Acute Occupancy,80.172,NaN
...,...,...,...,...,...,...,...
12027,Western,Yarmouth Regional Hospital,Regional Hospital,2023-05-01,Total Occupancy,80.876,NaN
12028,Western,Yarmouth Regional Hospital,Regional Hospital,2023-06-01,Total Occupancy,74.740,NaN
12029,Western,Yarmouth Regional Hospital,Regional Hospital,2023-07-01,Total Occupancy,81.483,NaN
12030,Western,Yarmouth Regional Hospital,Regional Hospital,2023-08-01,Total Occupancy,85.840,NaN


### Handle nulls

In [6]:
# hospital, type and CTAS have nulls
df.isnull().sum()

Zone               0
Hospital         791
Type             791
Date               0
Measure Name       0
Actual             0
CTAS            6911
dtype: int64

In [7]:
replaceNulls('Hospital', 'N/A')
replaceNulls('Type', 'N/A')
replaceNulls('CTAS', 'Blank')

### Create lookups, primary keys for dimensions 

In [8]:
zone_dict, zone_lup = lup_map(df, 'Zone')

type_dict, type_lup = lup_map(df, 'Type')

hospital_dict, hospital_lup = lup_map(df, 'Hospital')

measure_dict, measure_lup = lup_map(df, 'Measure Name')

ctas_dict, ctas_lup = lup_map(df, 'CTAS')

In [9]:
df['Date'].min(), df['Date'].max()

('2020-01-01', '2023-09-01')

In [10]:
# calendar dimension
cal_df = build_cal_dim('2020-01-01', '2024-04-01')
cal_df

,date,Year,Month,Day
0,2020-01-01,2020,1,1
1,2020-01-02,2020,1,2
2,2020-01-03,2020,1,3
3,2020-01-04,2020,1,4
4,2020-01-05,2020,1,5
...,...,...,...,...
1548,2024-03-28,2024,3,28
1549,2024-03-29,2024,3,29
1550,2024-03-30,2024,3,30
1551,2024-03-31,2024,3,31


In [11]:
cal_df['date'].nunique()

1553

### Column name edits

In [12]:
df.rename(columns={'Measure Name_ID':'Measure_ID'}, inplace=True)
df.rename(columns={'Measure Name':'Measure'}, inplace=True)

In [13]:
cal_df.rename(columns={'date':'Date'}, inplace=True)

In [14]:
df.columns

Index(['Zone', 'Hospital', 'Type', 'Date', 'Measure', 'Actual', 'CTAS',
       'Zone_ID', 'Type_ID', 'Hospital_ID', 'Measure_ID', 'CTAS_ID'],
      dtype='object')

### Dims and fact table builds

In [15]:
zone_df = df_dims(df, 'Zone_ID', 'Zone')
hospital_df = df_dims(df, 'Hospital_ID', 'Hospital')
type_df = df_dims(df, 'Type_ID', 'Type')
measure_df = df_dims(df, 'Measure_ID', 'Measure')
ctas_df = df_dims(df, 'CTAS_ID', 'CTAS')

In [16]:
zone_df

,Zone_ID,Zone
0,1,Central
186,2,Eastern
496,3,IWK
527,4,Northern
680,5,Western
3249,6,NS


In [17]:
fact_df = df[['Date', 'Zone_ID', 'Type_ID', 'Hospital_ID', 'Measure_ID', 'Actual', 'CTAS_ID']]

In [18]:
fact_df

,Date,Zone_ID,Type_ID,Hospital_ID,Measure_ID,Actual,CTAS_ID
0,2021-01-01,1,1,1,1,93.478,1
1,2021-04-01,1,1,1,1,80.797,1
2,2021-05-01,1,1,1,1,78.612,1
3,2021-06-01,1,1,1,1,75.525,1
4,2021-07-01,1,1,1,1,80.172,1
...,...,...,...,...,...,...,...
12027,2023-05-01,5,1,31,16,80.876,1
12028,2023-06-01,5,1,31,16,74.740,1
12029,2023-07-01,5,1,31,16,81.483,1
12030,2023-08-01,5,1,31,16,85.840,1


### Loading tables to MS SQL

In [19]:
#pip3 install pyodbc
from sqlalchemy import create_engine

In [20]:
def connect_mssql(user, password, host, database):
    """ Input your MS SQL user, password, host and database name.
    This will create the engine required fo connection. Returns engine.
    """
    engine = create_engine(f'mssql+pyodbc://{user}:{password}@{host}/{database}?driver=ODBC+Driver+17+for+SQL+Server')
    return engine
    
def load(df, table_name, schema):
    """ Input the dataframe you want to load, the table name you'd like for MS SQL,
    and which schema to load the table to. Schemas must be created in MS SQL ahead of time.
    This method uses the engine variable returned from connect_mssql function.
    """
    df.to_sql(table_name, engine, schema=schema, if_exists='replace', index=False)

In [21]:
# Create new database and schemas before connecting/loading

engine = connect_mssql('sa', 'password', 'localhost', 'HospitalData')

In [22]:
load(fact_df, 'HospitalData', 'f')

load(cal_df, 'Calendar', 'dim')

load(zone_df, 'Zone', 'dim')

load(type_df, 'Type', 'dim')

load(hospital_df, 'Hospital', 'dim')

load(measure_df, 'Measure', 'dim')

load(ctas_df, 'CTAS', 'dim')

In [ ]:
# Encode the password

from urllib.parse import quote_plus

password = quote_plus('pass@@word')